In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

# Launch DPS for tile_atl08.py

In [2]:
import os
import geopandas
import pandas as pd
import glob
import datetime
!pip install xmltodict
import xmltodict

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


  Using cached xmltodict-0.12.0-py2.py3-none-any.whl (9.2 kB)


In [3]:
def get_stack_fn(stack_list_fn, in_tile_num):
    # Find most recent topo/Landsat stack path for tile in list of stack paths from *tindex_master.csv
    all_stacks_df = pd.read_csv(stack_list_fn)
    stack_for_tile = all_stacks_df[all_stacks_df['location'].str.contains("_"+str(in_tile_num))]
    [print(i) for i in stack_for_tile.path.to_list()]
    stack_for_tile_fn = stack_for_tile.path.to_list()[0]
    if len(stack_for_tile)==0:
        stack_for_tile_fn = None
    return(stack_for_tile_fn)

# nmt added: code that returns df of landsat locations and tile number
# This is basically CountOutput.py
def get_stack_df(dps_dir, TYPE, dps_year):
    
    if "Landsat" in TYPE:
        root = f"/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/{dps_year}/"
        ends_with_str = "_dps.tif"
    if "Topo" in TYPE:
        root = f"/projects/my-private-bucket/dps_output/do_topo_stack_3-1-5_ubuntu/ops/{dps_year}/"
        ends_with_str = "_stack.tif"
    if "ATL08" in TYPE:
        root = f"/projects/my-private-bucket/dps_output/run_extract_ubuntu/ops/{dps_year}/"
        ends_with_str = "0m.csv"
            
    df = pd.DataFrame(columns=['location', 'tile_num'])

    for dir, subdir, files in os.walk(root):
        for fname in files:
            if fname.endswith(ends_with_str): 
                 
                tile_num = fname.split('_')[1]
                   
                if "ATL08" in TYPE:
                    df = df.append({'location':os.path.join(dir+"/", fname)},ignore_index=True)
                else:
                    df = df.append({'location':os.path.join(dir+"/", fname), 'tile_num':tile_num},ignore_index=True)
        
    return df

#### Set the names of the data frames to create

In [4]:
# Topo and Landsat tindex_master csvs from build_tindex_master.py
topo_tindex = "/projects/my-public-bucket/DPS_tile_lists/Topo_tindex_master.csv"
landsat_tindex = "/projects/my-public-bucket/DPS_tile_lists/Landsat_tindex_master.csv"

# Model-ready subset of tiles for which Topo and Landsat coincide
model_ready_tiles_topo = "/projects/my-public-bucket/DPS_tile_lists/model_ready_tiles_topo_paths.csv"
model_ready_tiles_landsat = "/projects/my-public-bucket/DPS_tile_lists/model_ready_tiles_landsat_paths.csv"

## Make the data frames from build_tindex_master.py csvs for Topo and Landsat tiles
python lib/build_tindex_master.py

In [5]:
if os.path.isfile(landsat_tindex) and os.path.isfile(topo_tindex):
    print('Reading existing...')
    ls8_df = pd.read_csv(landsat_tindex)
    topo_df = pd.read_csv(topo_tindex)
else:
    s3_stem = 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/nathanmthomas'
    local_stem = '/projects/my-private-bucket'

    ls8_root =  s3_stem + '/dps_output/do_landsat_stack_3-1-2_ubuntu'
    topo_root = s3_stem + '/dps_output/do_topo_stack_3-1-5_ubuntu'
    
    ls8_df = get_stack_df(ls8_root, "Landsat")
    topo_df = get_stack_df(topo_root, "Topo")
topo_df.head()

Reading existing...


,Unnamed: 0,local_path,tile_num
0,0,/projects/my-private-bucket/dps_output/do_topo...,421
1,1,/projects/my-private-bucket/dps_output/do_topo...,455
2,2,/projects/my-private-bucket/dps_output/do_topo...,456
3,3,/projects/my-private-bucket/dps_output/do_topo...,491
4,4,/projects/my-private-bucket/dps_output/do_topo...,492


In [6]:

topo_df = pd.read_csv(topo_tindex)
topo_df[topo_df.tile_num == 3457].local_path.tolist()[0].replace('/projects/my-private-bucket', 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/nathanmthomas')


'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/nathanmthomas/dps_output/do_topo_stack_3-1-5_ubuntu/ops/2021/07/23/23/32/27/934649/Copernicus_3457_covars_cog_topo_stack.tif'

## Get tile ids for which both Topo and Landsat stacks exist

In [9]:
# added by nmt: get filenames of co-incident landsat and topo
if False:
    topo_sub_df = pd.DataFrame(columns=['local_path','tile_num'])
    ls8_sub_df = pd.DataFrame(columns=['local_path','tile_num'])

    for i in range(len(ls8_df['tile_num'])):
        ls_tile_num = ls8_df['tile_num'][i]
        for j in range(len(topo_df['tile_num'])):
            topo_tile_num = topo_df['tile_num'][j]
            if ls_tile_num == topo_tile_num:
                # Only need to choose one, but we'll do 2 and then check
                ls8_sub_df = ls8_sub_df.append({'local_path':ls8_df['local_path'][i],'tile_num':ls8_df['tile_num'][i].astype(int)}, ignore_index=True)
                topo_sub_df = topo_sub_df.append({'local_path':topo_df['local_path'][j],'tile_num':topo_df['tile_num'][j].astype(int)}, ignore_index=True)

    #ls8_sub_df['tile_num'] = ls8_sub_df['tile_num'].astype(float, errors = 'raise')
    print(ls8_sub_df.head())
    print(topo_sub_df.head())
    print(len(ls8_sub_df),len(topo_sub_df))

    topo_sub_df.to_csv( model_ready_tiles_topo, index=False, encoding='utf-8-sig')
    ls8_sub_df.to_csv( model_ready_tiles_landsat, index=False, encoding='utf-8-sig')

#### Now you have a set of tile ids for which both Landsat and Topo stacks exist

In [10]:
topo_sub_df = pd.read_csv("/projects/my-public-bucket/DPS_tile_lists/model_ready_tiles_topo_paths.csv")
INPUT_TILE_NUM_LIST = topo_sub_df['tile_num'].values.astype(int).tolist()
len(INPUT_TILE_NUM_LIST)

4465

##### Test: get a subset of tile ids for test tiles (Norway and others in NA)

In [138]:
NORWAY_TILE_LIST = pd.read_csv('/projects/my-public-bucket/misc_files/norway_tiles.csv').layer.tolist()
DELTA_TILE_LIST = [3365,3366,3367,3458,3460,3353,3354,3355]
BONA_TILE_LIST  = [3270,3271,3272,3364,3456,3457,3458]
HEALY_TILE_LIST = [3456,3457,3458,3551,3553,3645,3646,3647]

INPUT_TEST_TILE_NUM_LIST = NORWAY_TILE_LIST + DELTA_TILE_LIST + BONA_TILE_LIST + HEALY_TILE_LIST

len(INPUT_TEST_TILE_NUM_LIST)

146

#### Read in the latest tindex and compare with a previous set of completed tiles to see which ones still need to be run

In [137]:
import numpy as np
tiles_completed = pd.read_csv('/projects/my-public-bucket/DPS_tile_lists/ATL08_filt_tindex_master.csv')
print(f'Tiles completed: {len(tiles_completed)}')
tile_nums_missing = np.setdiff1d(INPUT_TILE_NUM_LIST, tiles_completed.tile_num)
print(f'Tiles missing: {len(tile_nums_missing)}')
INPUT_TILE_NUM_LIST = tile_nums_missing.tolist()
len(INPUT_TILE_NUM_LIST)
#print(INPUT_TILE_NUM_LIST)

Tiles completed: 106
Tiles missing: 4325


4325

In [13]:
tindex_master_fn = f'/projects/shared-buckets/lduncanson/DPS_tile_lists/ATL08_tindex_master.csv'
tiles = pd.read_csv(tindex_master_fn)
len(tiles)

46166

In [139]:
TEST_DPS  = True

if TEST_DPS:
    DPS_INPUT_TILE_NUM_LIST = INPUT_TEST_TILE_NUM_LIST
    
    if False:
        !python /projects/icesat2_boreal/lib/build_tindex_master_v2.py -t ATL08_filt -y 2022 -m 03 -o /projects/test_dps
        t = pd.read_csv('/projects/test_dps/ATL08_filt_tindex_master.csv')
        COMPLETED_TILES = t.tile_num.to_list()
        NEED_TILES = list(set(DPS_INPUT_TILE_NUM_LIST) - set(COMPLETED_TILES))

        print(NEED_TILES)
        DPS_INPUT_TILE_NUM_LIST = NEED_TILES
    
else:
    DPS_INPUT_TILE_NUM_LIST = INPUT_TILE_NUM_LIST
    
print(f"# of tiles to run: {len(DPS_INPUT_TILE_NUM_LIST)}\n", DPS_INPUT_TILE_NUM_LIST)


# of tiles to run: 146
 [131, 132, 133, 4, 5, 6, 7, 270, 271, 9, 10, 272, 273, 11, 274, 12, 13, 275, 14, 16, 17, 18, 19, 20, 151, 152, 21, 153, 154, 416, 417, 25, 418, 26, 419, 27, 28, 29, 30, 31, 296, 297, 298, 36, 299, 37, 300, 38, 39, 301, 40, 41, 42, 174, 43, 175, 176, 177, 178, 48, 49, 50, 51, 52, 53, 54, 59, 60, 322, 61, 323, 62, 324, 325, 63, 326, 64, 327, 65, 197, 328, 198, 199, 200, 201, 72, 73, 74, 75, 76, 77, 26025, 221, 352, 222, 91, 353, 223, 92, 354, 224, 93, 355, 94, 356, 225, 357, 110, 111, 112, 26574, 113, 246, 247, 248, 249, 250, 384, 385, 386, 387, 388, 130, 3365, 3366, 3367, 3458, 3460, 3353, 3354, 3355, 3270, 3271, 3272, 3364, 3456, 3457, 3458, 3456, 3457, 3458, 3551, 3553, 3645, 3646, 3647]


#### Customize the DPS run: set up the parameters dictionary

In [140]:
# Norway test 01
# Just include sol_el so we can use sol_el < 5
in_param_dict_norway01 = {
                        'in_tile_num': '',
                        'in_tile_fn': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                        'in_tile_layer': 'boreal_tiles_v003',
                        'csv_list_fn': 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_tindex_master.csv',
                        'topo_stack_list_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv',
                        'landsat_stack_list_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Landsat_tindex_master.csv',
                        'years_list': '2019 2020 2021',
                        'user_stacks': 'nathanmthomas',
                        'user_atl08': 'lduncanson',
                        'thresh_sol_el': 5,
                        'v_ATL08': 4,
                        'minmonth': 6,
                        'maxmonth': 9,
                        'LC_filter': False
    }
# Norway test 02
# Use v005 ATL08, which will apply lc-based thresholds, extend to all months
# NOTE!! make sure you manually update to use the correct filter in tile_atl08.py
in_param_dict_norway02 = {
                        'in_tile_num': '',
                        'in_tile_fn': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                        'in_tile_layer': 'boreal_tiles_v003',
                        'csv_list_fn': 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_tindex_master.csv',
                        'topo_stack_list_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv',
                        'landsat_stack_list_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Landsat_tindex_master.csv',
                        'years_list': '2019 2020 2021',
                        'user_stacks': 'nathanmthomas',
                        'user_atl08': 'lduncanson',
                        'thresh_sol_el': 5,
                        'v_ATL08': 5,
                        'minmonth': 4,
                        'maxmonth': 10,
                        'LC_filter': True
    }

In [141]:
in_param_dict = in_param_dict_norway02
in_param_dict

{'in_tile_num': '',
 'in_tile_fn': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
 'in_tile_layer': 'boreal_tiles_v003',
 'csv_list_fn': 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_tindex_master.csv',
 'topo_stack_list_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv',
 'landsat_stack_list_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Landsat_tindex_master.csv',
 'years_list': '2019 2020 2021',
 'user_stacks': 'nathanmthomas',
 'user_atl08': 'lduncanson',
 'thresh_sol_el': 5,
 'v_ATL08': 5,
 'minmonth': 4,
 'maxmonth': 10,
 'LC_filter': True}

## Run a DPS job across the list

In [142]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):

    DPS_num = i+1
    IDENTIFIER = 'run_tile_atl08'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-8gb'
    
    in_param_dict['in_tile_num'] = INPUT_TILE_NUM
    
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='master',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )
    
    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE
    
    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 25, 50, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000, len_input_list]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 

# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df

# of input tiles for DPS: 146
DPS run #: 1	| tile num: 131	| submit status: success	| job id: 2a8b4f23-1890-4e17-96cb-0b4d014ea3a0
DPS run #: 25	| tile num: 151	| submit status: success	| job id: fc80c2e2-1ff4-41c4-b37d-18d950788e35
DPS run #: 50	| tile num: 301	| submit status: success	| job id: ffeb0c34-6dd6-4751-afde-1dbff697e84e
DPS run #: 100	| tile num: 354	| submit status: success	| job id: e31b9914-e20a-4a6e-85bd-42ff9e3ad238
DPS run #: 146	| tile num: 3647	| submit status: success	| job id: 51db777e-4c83-40e5-bf1d-bf47ca352121
Current time:	202203120338
CPU times: user 1.73 s, sys: 139 ms, total: 1.87 s
Wall time: 34.7 s


,status,http_status_code,job_id,dps_num,tile_num,submit_time,dbs_job_hour,algo_id,user,worker_type
0,success,200,2a8b4f23-1890-4e17-96cb-0b4d014ea3a0,1,131,2022-03-12 03:37:38.360146,3,run_tile_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,a99f2eb7-fcb4-4542-84c1-81041f4c22ba,2,132,2022-03-12 03:37:38.444376,3,run_tile_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,22a0fb26-0b03-436d-9ec8-f05537b2fa06,3,133,2022-03-12 03:37:38.546460,3,run_tile_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,dd88b42e-4c4c-4558-a24e-26b742234a99,4,4,2022-03-12 03:37:38.723344,3,run_tile_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,816d633a-fa22-4dc5-86bc-dde852eef2b2,5,5,2022-03-12 03:37:38.942003,3,run_tile_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
...,...,...,...,...,...,...,...,...,...,...
0,success,200,ef733b08-8708-43ef-9cd2-6903333b9b59,142,3551,2022-03-12 03:38:11.665981,3,run_tile_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,e9bf8480-c0b5-42d4-bbdf-75fb5af0930a,143,3553,2022-03-12 03:38:11.910037,3,run_tile_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,c7304ea5-04ec-49f9-ba2d-52871f60fa3c,144,3645,2022-03-12 03:38:12.131826,3,run_tile_atl08_ubuntu,lduncanson,maap-dps-worker-8gb
0,success,200,43ada2ed-d001-4484-8543-dc6b65c82511,145,3646,2022-03-12 03:38:12.385556,3,run_tile_atl08_ubuntu,lduncanson,maap-dps-worker-8gb


After almost any DPS job, you have to assess what succeeded and failed. This involves:
1. building a table of job status based on job ids captured in the job_results_df from the DPS run chunk (this takes 40 mins for ~47k jobs) --> this tells you how many jobs failed
2. merging the job status table with the job results df --> this tells you which specific granules (or tile nums) failed
3. building another input list of granules for a follow-up DPS
## Assess DPS results
Build a table of job status based on job id - how many jobs failed?

In [156]:
%%time
def BUILD_TABLE_JOBSTATUS(submit_results_df):
    import xmltodict
    df = pd.concat([pd.DataFrame(xmltodict.parse(maap.getJobStatus(job_id).content)).transpose() for job_id in submit_results_df.job_id.to_list()])
    return df

job_status_df = BUILD_TABLE_JOBSTATUS(submit_results_df)
#print(job_status_df.head())

num_jobs = submit_results_df.shape[0]
z = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='wps:JobID')

print(f'Count total jobs:\t{num_jobs}')
print(f"Count pending jobs:\t{z[z['wps:Status'] =='Accepted'].shape[0]}")
print(f"Count running jobs:\t{z[z['wps:Status'] =='Running'].shape[0]}")
print(f"Count succeeded jobs:\t{z[z['wps:Status'] =='Succeeded'].shape[0]}")
print(f"Count failed jobs:\t{z[z['wps:Status'] =='Failed'].shape[0]}")
print(f"% of failed jobs:\t{round(z[z['wps:Status'] =='Failed'].shape[0] / ( z[z['wps:Status'] =='Failed'].shape[0] + z[z['wps:Status'] =='Succeeded'].shape[0] ), 4) * 100}")

Count total jobs:	146
Count pending jobs:	0
Count running jobs:	0
Count succeeded jobs:	0
Count failed jobs:	146
% of failed jobs:	100.0
CPU times: user 1.51 s, sys: 143 ms, total: 1.66 s
Wall time: 4.1 s


In [128]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Succeeded'].iloc[1].job_id).content)

OrderedDict([('wps:Result',
              OrderedDict([('@xmlns:ows', 'http://www.opengis.net/ows/2.0'),
                           ('@xmlns:schemaLocation',
                            'http://schemas.opengis.net/wps/2.0/wps.xsd'),
                           ('@xmlns:wps', 'http://www.opengis.net/wps/2.0'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('wps:JobID',
                            '06d93201-6296-42b4-a039-20f6ad0956d4'),
                           ('wps:Output',
                            OrderedDict([('@id',
                                          'output-2022-03-12T01:38:29.587754'),
                                         ('wps:Data',
                                          ['http://maap-ops-workspace.s3-website-us-west-2.amazonaws.com/lduncanson/dps_output/run_tile_atl08_ubuntu/master/2022/03/12/01/38/29/587754',
                                           's3://s3.us

In [151]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Failed'].iloc[0].job_id).content)

OrderedDict([('wps:Result',
              OrderedDict([('@xmlns:ows', 'http://www.opengis.net/ows/2.0'),
                           ('@xmlns:schemaLocation',
                            'http://schemas.opengis.net/wps/2.0/wps.xsd'),
                           ('@xmlns:wps', 'http://www.opengis.net/wps/2.0'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('wps:JobID',
                            '2a8b4f23-1890-4e17-96cb-0b4d014ea3a0'),
                           ('wps:Output',
                            OrderedDict([('@id', 'traceback'),
                                         ('wps:Data',
                                          "activate does not accept more than one argument:\n['/app/icesat2_boreal/dps/alg_2-4/run_tile_atl08.sh', '131', 'boreal_tiles_v003', 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_tindex_master.csv', 's3://maap-ops-workspace/shared/nathanmthomas/

In [75]:
!python /projects/icesat2_boreal/lib/build_tindex_master_v2.py -t ATL08_filt -y 2022 -m 3 -o /projects/test_dps

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

Building a list of tiles:  ATL08_filt

Output dir:  /projects/test_dps
/projects/icesat2_boreal/lib/build_tindex_master_v2.py:154: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tile_num'] = df['file'].str.split('_', expand=True)[7].str.replace('.csv','')
# of duplicate tiles: 4
Final # of tiles: 103
df shape :                                              s3_path  ... tile_num
0  s3://maap-ops-workspace/lduncanson/dps_output/...  ...     0054
2  s3://maap-ops-workspace/lduncanson/dps_output/...  ...     0043
4  s3://maap-ops-workspace/lduncanson/dps_output/...  ...     0031
6  s3://maap-ops-workspace/lduncanson/dps_output/...  ...     0

[387, 388, 5, 6, 7, 10, 14, 16, 275, 21, 25, 27, 28, 29, 416, 37, 38, 296, 26025, 299, 300, 301, 48, 177, 50, 178, 60, 198, 3270, 72, 326, 327, 328, 26574, 354, 355, 356, 357, 247]


In [153]:
%%time
TILE_NUM = 131 #NEED_TILES[6]
args = f"\
-LC_filter True \
--extract_covars \
--do_30m \
--do_dps \
-years_list 2019 2020 2021 \
-o /projects/my-public-bucket/atl08_filt_covar_tiles \
-in_tile_num {TILE_NUM} \
-in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg \
-in_tile_layer boreal_tiles_v003 \
-in_tile_id_col tile_num \
-csv_list_fn /projects/shared-buckets/lduncanson/DPS_tile_lists/ATL08_tindex_master.csv \
-topo_stack_list_fn /projects/shared-buckets/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv \
-landsat_stack_list_fn /projects/shared-buckets/nathanmthomas/DPS_tile_lists/Landsat_tindex_master.csv \
-user_stacks nathanmthomas \
-user_atl08 lduncanson \
-thresh_sol_el 5 \
-v_ATL08 5 -minmonth 4 -maxmonth 10"
print(args)
!python /projects/icesat2_boreal/lib/tile_atl08.py $args

-LC_filter True --extract_covars --do_30m --do_dps -years_list 2019 2020 2021 -o /projects/my-public-bucket/atl08_filt_covar_tiles -in_tile_num 131 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_layer boreal_tiles_v003 -in_tile_id_col tile_num -csv_list_fn /projects/shared-buckets/lduncanson/DPS_tile_lists/ATL08_tindex_master.csv -topo_stack_list_fn /projects/shared-buckets/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv -landsat_stack_list_fn /projects/shared-buckets/nathanmthomas/DPS_tile_lists/Landsat_tindex_master.csv -user_stacks nathanmthomas -user_atl08 lduncanson -thresh_sol_el 5 -v_ATL08 5 -minmonth 4 -maxmonth 10
/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

Land cover filtering set t